# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary 
import math #The Python math module
from scipy import stats #The SciPy stats module
from statistics import mean #The statistics module's mean function

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world we would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

We will not be paying for access to the index provider's API rather there is a static version of the S&P 500 constituents available here.
[Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). 

Move this file to where this Jupyter Notebook file is located.

Now we import these stocks.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now we import our IEX Cloud API token.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to local Git repository. We'll be using a sandbox API token in this project, which means that the data we'll use is randomly-generated.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2291050069166,
 'week52high': 150.6,
 'week52low': 91.78,
 'week52highSplitAdjustOnly': 149.59,
 'week52lowSplitAdjustOnly': 91.4,
 'week52change': 0,
 'sharesOutstanding': 17034691700,
 'float': 0,
 'avg10Volume': 67360724,
 'avg30Volume': 76046457,
 'day200MovingAvg': 129.55,
 'day50MovingAvg': 131.48,
 'employees': 147854,
 'ttmEPS': 4.6,
 'ttmDividendRate': 0.8368337763357195,
 'dividendYield': 0.006238573670999693,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-26',
 'nextEarningsDate': '2021-07-19',
 'peRatio': 31.02365749289955,
 'beta': 1.6176404245412275,
 'maxChangePercent': 53.3590525755906,
 'year5ChangePercent': 5.372894564320931,
 'year2ChangePercent': 1.888100685903372,
 'year1ChangePercent': 0.5174839185931464,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.03582093851655092,
 'month3ChangePercent': 0.12610486720894717,
 'month1ChangePercent': 0.10222422176430997,
 'day30ChangePercent': 0.09921458908855202,
 'day5Chang

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation.
Here is an example.

In [4]:
data['year1ChangePercent']

0.5174839185931464

## Executing A Batch API Call & Building Our DataFrame

Batch API calls are one of the easiest ways to improve the performance of code.

This is because HTTP requests are typically one of the slowest components of a script.

Batch API calls are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,148.100,0.694725,N/A
1,AAL,21.955,0.647075,N/A
2,AAP,217.240,0.466885,N/A
3,AAPL,137.976,0.52076,N/A
4,ABBV,118.250,0.208082,N/A
...,...,...,...,...
500,YUM,119.123,0.349841,N/A
501,ZBH,167.880,0.367071,N/A
502,ZBRA,531.330,1.077168,N/A
503,ZION,55.220,0.615149,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,76.660,3.891245,N/A
1,TPR,42.400,2.371536,N/A
2,FCX,38.130,2.284709,N/A
3,DVN,31.270,1.778791,N/A
4,GPS,35.140,1.760501,N/A
5,KSS,56.990,1.693019,N/A
6,SIVB,585.780,1.656203,N/A
7,MGM,44.800,1.614542,N/A
8,IVZ,27.310,1.601596,N/A
9,MOS,33.610,1.573158,N/A


## Calculating the Number of Shares to Buy



In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \nTry again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000000000000
10000000000000000


In [5]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in final_dataframe.index:
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

NameError: name 'portfolio_size' is not defined

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards suddenly.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.790,N/A,0.692509,N/A,0.252377,N/A,0.165805,N/A,0.071783,N/A,N/A
1,AAL,22.455,N/A,0.651992,N/A,0.350531,N/A,-0.11765,N/A,-0.129905,N/A,N/A
2,AAP,214.511,N/A,0.470335,N/A,0.312799,N/A,0.126336,N/A,0.088808,N/A,N/A
3,AAPL,138.270,N/A,0.5326,N/A,0.036558,N/A,0.123834,N/A,0.101616,N/A,N/A
4,ABBV,114.840,N/A,0.205514,N/A,0.077354,N/A,0.054216,N/A,-0.005166,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,116.840,N/A,0.357793,N/A,0.072463,N/A,0.07078,N/A,-0.042319,N/A,N/A
501,ZBH,166.490,N/A,0.356243,N/A,0.04731,N/A,0.00613,N/A,-0.045267,N/A,N/A
502,ZBRA,541.020,N/A,1.09417,N/A,0.391984,N/A,0.09178,N/A,0.065438,N/A,N/A
503,ZION,55.900,N/A,0.603474,N/A,0.235024,N/A,-0.0336,N/A,-0.089982,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

We can use pandas' `isnull` method to identify missing data:

In [11]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
118,CTL,11.00,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
165,ETFC,49.69,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
326,MYL,15.87,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
327,NBL,8.60,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this project, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [12]:
for column in ['One-Year Price Return', 'Six-Month Price Return','Three-Month Price Return', 'One-Month Price Return']:
    hqm_dataframe[column].fillna(hqm_dataframe[column].min(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [13]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [23]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
            

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0     0.84
1     0.96
2     0.78
3     0.72
4     0.46
5     0.36
6     0.62
7     0.32
8     0.86
9     0.56
10    0.66
11     1.0
12    0.92
13    0.52
14    0.28
15     0.8
16     0.7
17    0.44
18    0.16
19    0.48
20    0.74
21    0.58
22    0.76
23    0.64
24     0.9
25    0.34
26     0.5
27    0.24
28    0.98
29     0.4
30     0.1
31    0.42
32     0.2
33    0.06
34    0.68
35    0.04
36    0.14
37    0.82
38    0.08
39    0.18
40     0.6
41    0.38
42    0.88
43    0.26
44    0.12
45    0.02
46    0.54
47    0.94
48     0.3
49    0.22
Name: One-Year Return Percentile, dtype: object
0     0.96
1     0.94
2      1.0
3      0.7
4     0.56
5      0.8
6     0.68
7     0.92
8     0.66
9     0.46
10    0.38
11    0.98
12    0.88
13    0.16
14    0.84
15    0.52
16    0.62
17     0.5
18    0.22
19    0.24
20    0.54
21    0.58
22    0.18
23    0.72
24    0.26
25    0.44
26    0.64
27     0.9
28    0.86
29     0.6
30    0.28
31    0.42
32     0.2
33    0.74
34     0.4
35    0.36
36    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FANG,102.660,0,1.366102,0.84,0.985412,0.96,0.290089,0.88,0.180999,0.92,0.983168
1,DVN,30.294,0,1.740894,0.96,0.895106,0.94,0.344856,0.98,0.106174,0.76,0.982178
2,MRO,14.770,0,1.259153,0.78,1.064868,1.0,0.289983,0.86,0.126710,0.84,0.974257
3,NVDA,842.190,0,1.126708,0.72,0.540768,0.7,0.505638,1.0,0.237512,0.98,0.973267
4,EXR,172.780,0,0.828751,0.46,0.441277,0.56,0.254365,0.78,0.105262,0.74,0.923267
5,FTNT,247.131,0,0.755197,0.36,0.613090,0.8,0.301463,0.9,0.092437,0.7,0.921782
6,IT,256.340,0,1.019260,0.62,0.536522,0.68,0.338287,0.96,0.045374,0.4,0.916832
7,OXY,33.400,0,0.739436,0.32,0.820825,0.92,0.183441,0.4,0.210149,0.96,0.914356
8,DXC,41.600,0,1.389407,0.86,0.514764,0.66,0.245823,0.74,0.027063,0.24,0.906436
9,WAT,357.060,0,0.951138,0.56,0.407643,0.46,0.220059,0.58,0.075411,0.66,0.90495


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [15]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.790,N/A,0.692509,0.742574,0.252377,0.677228,0.165805,0.847525,0.071783,0.889109,0.789109
1,AAL,22.455,N/A,0.651992,0.70297,0.350531,0.843564,-0.117650,0.027723,-0.129905,0.017822,0.39802
2,AAP,214.511,N/A,0.470335,0.524752,0.312799,0.79802,0.126336,0.732673,0.088808,0.924752,0.74505
3,AAPL,138.270,N/A,0.532600,0.584158,0.036558,0.192079,0.123834,0.728713,0.101616,0.938614,0.610891
4,ABBV,114.840,N/A,0.205514,0.2,0.077354,0.277228,0.054216,0.483168,-0.005166,0.528713,0.372277
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,116.840,N/A,0.357793,0.392079,0.072463,0.265347,0.070780,0.558416,-0.042319,0.259406,0.368812
501,ZBH,166.490,N/A,0.356243,0.390099,0.047310,0.213861,0.006130,0.283168,-0.045267,0.235644,0.280693
502,ZBRA,541.020,N/A,1.094170,0.922772,0.391984,0.891089,0.091780,0.631683,0.065438,0.865347,0.827723
503,ZION,55.900,N/A,0.603474,0.657426,0.235024,0.641584,-0.033600,0.122772,-0.089982,0.061386,0.370792


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [17]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)

<ipython-input-17-3227035fa198>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FANG,102.660,N/A,1.366102,0.966337,0.985412,0.99604,0.290089,0.982178,0.180999,0.988119,0.983168
1,DVN,30.294,N/A,1.740894,0.992079,0.895106,0.994059,0.344856,0.99604,0.106174,0.946535,0.982178
2,MRO,14.770,N/A,1.259153,0.956436,1.064868,1.0,0.289983,0.980198,0.126710,0.960396,0.974257
3,NVDA,842.190,N/A,1.126708,0.930693,0.540768,0.966337,0.505638,1.0,0.237512,0.99604,0.973267
4,EXR,172.780,N/A,0.828751,0.843564,0.441277,0.938614,0.254365,0.966337,0.105262,0.944554,0.923267
5,FTNT,247.131,N/A,0.755197,0.79604,0.613090,0.978218,0.301463,0.986139,0.092437,0.926733,0.921782
6,IT,256.340,N/A,1.019260,0.90495,0.536522,0.964356,0.338287,0.994059,0.045374,0.80396,0.916832
7,OXY,33.400,N/A,0.739436,0.786139,0.820825,0.990099,0.183441,0.887129,0.210149,0.994059,0.914356
8,DXC,41.600,N/A,1.389407,0.972277,0.514764,0.958416,0.245823,0.960396,0.027063,0.734653,0.906436
9,WAT,357.060,N/A,0.951138,0.885149,0.407643,0.90495,0.220059,0.930693,0.075411,0.89901,0.90495


## Calculating the Number of Shares to Buy


In [18]:
portfolio_input()

Enter the value of your portfolio:100


In [19]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FANG,102.660,0,1.366102,0.966337,0.985412,0.99604,0.290089,0.982178,0.180999,0.988119,0.983168
1,DVN,30.294,0,1.740894,0.992079,0.895106,0.994059,0.344856,0.99604,0.106174,0.946535,0.982178
2,MRO,14.770,0,1.259153,0.956436,1.064868,1.0,0.289983,0.980198,0.126710,0.960396,0.974257
3,NVDA,842.190,0,1.126708,0.930693,0.540768,0.966337,0.505638,1.0,0.237512,0.99604,0.973267
4,EXR,172.780,0,0.828751,0.843564,0.441277,0.938614,0.254365,0.966337,0.105262,0.944554,0.923267
5,FTNT,247.131,0,0.755197,0.79604,0.613090,0.978218,0.301463,0.986139,0.092437,0.926733,0.921782
6,IT,256.340,0,1.019260,0.90495,0.536522,0.964356,0.338287,0.994059,0.045374,0.80396,0.916832
7,OXY,33.400,0,0.739436,0.786139,0.820825,0.990099,0.183441,0.887129,0.210149,0.994059,0.914356
8,DXC,41.600,0,1.389407,0.972277,0.514764,0.958416,0.245823,0.960396,0.027063,0.734653,0.906436
9,WAT,357.060,0,0.951138,0.885149,0.407643,0.90495,0.220059,0.930693,0.075411,0.89901,0.90495


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.

In [20]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [22]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 27, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output


In [ ]:
writer.save()